<a href="https://colab.research.google.com/github/praveen555/IMI-BIGDATA--CHALLENGE/blob/main/KYC_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install python-levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.2 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import re 
import matplotlib.pyplot as plt
from datetime import datetime, date
import numpy as np
from google.colab import drive
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
drive.mount('/content/drive')
path='/content/drive/MyDrive/Colab Notebooks/IMI_BigData_Challenge/Raw data files'
path_2='/content/drive/MyDrive/Colab Notebooks/IMI_BigData_Challenge/Task_1_Data'
path_edge=os.path.join(path,'UofT_edges.csv')
path_nodes=os.path.join(path,'UofT_nodes.csv')
path_occupation_risk=os.path.join(path,'UofT_occupation_risk.csv')
path_name_list='/content/drive/MyDrive/Colab Notebooks/IMI_BigData_Challenge/name_list.csv'
path_clean_customer=os.path.join(path_2,'clean_customer_names.csv')
path_clean_sanctions=os.path.join(path_2,'clean_sanction_names.csv')

Mounted at /content/drive


In [5]:
customer_names=pd.read_csv('ScotiaBank_names.csv')

In [6]:
customer_names

,NAME,BIRTH_DT
0,young marie mildren coleman,1981-09-01
1,mark stupar lecy,1994-02-21
2,dean glasper wendel reeves,1962-11-16
3,hulsey linda rauth,1998-06-20
4,carolyn washington roberts,1942-01-24
...,...,...
999251,bret dove gainey,1939-01-16
999252,anthony ray montano gomez,1970-12-13
999253,bernadette schofield pace mile,1958-09-09
999254,wasson russell terrell,1983-09-15


In [9]:
sanction_names=pd.read_csv('clean_sanction_name.csv')

In [10]:
sanction_names

,name,aliases,birth_date
0,kolyvanov egor,Колыванов Егор,1980-11-15
1,shipov sergei yurievich,Шипов Сергей Юрьевич,1966-04-17
2,egorov ivan mikhailovich,Егоров Иван Михайлович,1961-01-21
3,goreslavsky alexey sergeyevich,Гореславский Алексей Сергеевич,1977-07-13
4,samoilova natalya vladimirovna,Самойлова Наталья Владимировна,1987-06-24
...,...,...,...
206684,rustam abdulsaidovich geremeev,Рустам Абдулсаидович ​Геремеев;​Geremeev Rusta...,1982-04-26
206685,khedi abdulsaidovna geremeeva,Хеди Абдулсаидовна ​Геремеева;​Geremeeva Hedi ...,NaN
206686,izumrud muzhayevna mugutdinova,Изумруд Мужайевна​ ​Мугутдинова;​Mugutdinova I...,1954-02-23
206687,amkhad kharalievich khatataev,Амхад Харалиевич​ ​Хататаев;​Hatataev Amhad Ha...,NaN


In [11]:
sanction_names.rename(columns={'birth_date':"BIRTH_DT"},inplace=True)

In [12]:
sanction_names.head(2)

,name,aliases,BIRTH_DT
0,kolyvanov egor,Колыванов Егор,1980-11-15
1,shipov sergei yurievich,Шипов Сергей Юрьевич,1966-04-17


In [13]:
df=pd.merge(customer_names,sanction_names,on='BIRTH_DT')

In [14]:
df

,NAME,BIRTH_DT,name,aliases
0,young marie mildren coleman,1981-09-01,adrian cusiac,NaN
1,young marie mildren coleman,1981-09-01,petrovic zelimir,Zelimir Petrovic
2,young marie mildren coleman,1981-09-01,josé ignacio paliza,Jose Ignacio Paliza;Jose Paliza;José Ignacio P...
3,young marie mildren coleman,1981-09-01,inger lise hansen,NaN
4,young marie mildren coleman,1981-09-01,matt warman,"Matthew Robert Warman;Warman, Matt;مات وارمان;..."
...,...,...,...,...
5000643,june vick fultz,1921-04-21,pierre teuscher,"Teuscher, Pierre"
5000644,june vick fultz,1921-04-21,nicolaus dreyer,Dreyer!;Nikolaus Dreyer;نيكولاوس درير
5000645,nick tamayo turner,1925-08-31,paolo tringali,NaN
5000646,nick tamayo turner,1925-08-31,dewey l hill,Dewey Lewis Hill;ديوى ال. هيل;ديوي إل. هيل;دیو...


In [17]:
def fuzzyscore(col):
  return fuzz.token_sort_ratio(col['NAME'],col['name'])



In [18]:
df['score']=df.apply(fuzzyscore,axis=1)

In [19]:
high_risk_names=df[(df['score']>=75)]

In [20]:
high_risk_names.shape

(43, 5)

In [21]:
high_risk_names.sort_values('score',ascending=False,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [22]:
high_risk_names

,NAME,BIRTH_DT,name,aliases,score
3716386,samantha lewthwaite,1983-12-05,samantha lewthwaite,Bílá vdova;SAMANTHA LOUISE LEWTHWAITE;Samantha...,100
376833,hlaing min aung,1956-07-03,min aung hlaing,"El Pinochet Birmano;HLAING, Min Aung;Jeneral K...",100
3506215,o jong gil,1962-08-30,jong gil o,Jong Gil O;O Jong Gil,100
857116,zheynova marina nikolaevna,1985-02-15,marina nikolaevna zheynova,Marina Nikolaevna Zheynova;Marina Nikolajevna ...,100
2853650,paul franklin watson,1950-12-02,paul franklin watson,Paul Franklin Watson;Paul Watson;Пол Ватсон;По...,100
2645979,alexey alexeyevich gromov,1960-05-31,alexey alexeyevich gromov,A. Gromov;Aleksei Gromov;Aleksej Aleksejevitsj...,100
1439126,bezrukov sergey vitalyevich,1973-10-18,sergey vitalyevich bezrukov,BEZRUKOV Sergei Vitalevich;Bezrukov Sergey Vit...,100
4197460,tsertsel ivan stanislavavich,1966-09-08,ivan stanislavavich tertel,Ivan Stanislavavich TSERTSEL;Ivan Stanislavavi...,96
2066226,trinidad y ramirez angelo,1978-03-20,angelo ramirez trinidad,ANGELO RAMIREZ TRINIDAD;Abdukahlil;Abdul Khali...,96
3306622,rakhim azizboevich azimov,1964-08-16,rahim azizboevich asimov,"ASIMOV, Rahim Azizboyevich;AZIMOV Rakhim Azizb...",94


In [ ]:
#high_risk_names.to_csv('high_risk_names_v2.csv')

## secondary process ideas ? 


